In [6]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
print(torch.__version__)  # PyTorch 버전 출력
print(torch.cuda.is_available())  # CUDA 사용 가능 여부 확인

2.5.1
True


In [7]:

# 1단계: 데이터 로드 및 전처리
file_path = '/home/gyuha_lee/Opensource_dev/get_information/mine/Airline_review.csv'  # 데이터셋 경로를 업데이트하세요
data = pd.read_csv(file_path)

# 기본 전처리
data['Review'] = data['Review'].str.lower().str.replace('[^\w\s]', '')  # 리뷰를 소문자로 변환하고 특수문자 제거
data['Type Of Traveller'] = data['Type Of Traveller'].fillna('Unknown')  # 여행 유형의 결측값을 'Unknown'으로 채움


In [8]:

# 2단계: BERT를 이용한 감성 분석
# GPU 설정 및 모델 로드
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=model_name,
    device=0 if torch.cuda.is_available() else -1  # GPU 사용 설정
)

# 긴 리뷰를 최대 길이로 잘라내는 함수 정의
def truncate_review(review, max_length=512):
    return " ".join(review.split()[:max_length])

# 긴 리뷰를 자르고 감성 분석 수행
data['Truncated_Review'] = data['Review'].apply(lambda x: truncate_review(x, max_length=512))

# 진행률 출력하면서 감성 분석 수행
sentiments = []
total_reviews = len(data)
progress_interval = max(total_reviews // 100, 1)  # 진행률 출력 간격 (1% 단위)

for i, review in enumerate(data['Truncated_Review']):
    try:
        result = sentiment_analyzer(review)[0]['label']
        sentiments.append(result)
    except Exception as e:
        sentiments.append(None)  # 에러 발생 시 None 처리
    
    # 진행 상태 출력
    if (i + 1) % progress_interval == 0 or (i + 1) == total_reviews:
        progress = (i + 1) / total_reviews * 100
        print(f"Progress: {progress:.2f}% ({i + 1}/{total_reviews})", end="\r")

# 감성 분석 결과를 데이터에 추가
data['Sentiment'] = sentiments

# 감성 점수를 숫자로 매핑
sentiment_mapping = {
    '1 star': -2, '2 stars': -1, '3 stars': 0, '4 stars': 1, '5 stars': 2
}
data['Sentiment_Score'] = data['Sentiment'].map(sentiment_mapping)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Token indices sequence length is longer than the specified maximum sequence length for this model (575 > 512). Running this sequence through the model will result in indexing errors


In [9]:

# 3단계: 추천 함수 정의
def recommend_airline(user_type, preference, seat_type):
    """
    사용자 입력 및 리뷰 데이터를 분석하여 항공사를 추천합니다.
    
    Args:
        user_type (str): 여행객 유형 (예: 'Solo Leisure', 'Business')
        preference (str): 주요 선호 요소 (예: 'Service', 'Seat Comfort')
        seat_type (str): 원하는 좌석 유형 (예: 'Economy Class')
        
    Returns:
        pd.DataFrame: 추천 항공사 상위 목록
    """
    # 사용자 입력에 맞는 데이터를 필터링
    filtered_data = data[
        (data['Type Of Traveller'] == user_type) &
        (data['Seat Type'] == seat_type)
    ]
    
    # 선호 요소와 감성 점수를 기반으로 점수 계산
    preference_columns = {
        'Service': 'Cabin Staff Service',
        'Seat Comfort': 'Seat Comfort',
        'Food': 'Food & Beverages'
    }
    filtered_data['Preference_Score'] = filtered_data[preference_columns[preference]].fillna(0)
    filtered_data['Total_Score'] = filtered_data['Sentiment_Score'] + filtered_data['Preference_Score']
    
    # 항공사별로 점수를 집계하고 정렬
    recommendations = (
        filtered_data.groupby('Airline Name')['Total_Score']
        .mean()
        .sort_values(ascending=False)
        .reset_index()
    )
    return recommendations.head(5)


In [10]:

# 4단계: 예시 실행
# 사용자 입력
user_type = 'Solo Leisure'  # 예: 혼자 여행하는 경우
preference = 'Service'      # 예: 서비스 품질을 중시
seat_type = 'Economy Class' # 예: 이코노미 클래스 선호

# 추천 결과 도출
top_airlines = recommend_airline(user_type, preference, seat_type)


/tmp/ipykernel_2206/2821009805.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Preference_Score'] = filtered_data[preference_columns[preference]].fillna(0)
/tmp/ipykernel_2206/2821009805.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Total_Score'] = filtered_data['Sentiment_Score'] + filtered_data['Preference_Score']


In [11]:

# 결과 출력
print("추천 항공사 상위 목록:")
print(top_airlines)


추천 항공사 상위 목록:
          Airline Name  Total_Score
0         Air Zimbabwe          7.0
1    Skytrans Airlines          7.0
2  Air Iceland Connect          7.0
3            Air Busan          7.0
4          Bassaka Air          7.0
